<a href="https://colab.research.google.com/github/AdrianDiazGaxiola/Proyecto-Final/blob/main/Sequ%C3%ADas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Este programa se empleará para limpiar el DataFrame con los datos de las sequías en México

In [1]:
!pip install numpy
!pip install pandas
!pip install openpyxl

Se copia el archivo de github con los datos a un dataframe

In [2]:
import pandas as pd

url = "https://raw.githubusercontent.com/AdrianDiazGaxiola/Proyecto-Final/main/Data/Sequ%C3%ADas.csv"
Seq = pd.read_csv(url, encoding='ISO-8859-1')

print(Seq)

      Clave del Municipio  ... Total (D0 a D4)
0                  1001.0  ...            65.0
1                  1002.0  ...            64.0
2                  1003.0  ...            71.0
3                  1004.0  ...            61.0
4                  1005.0  ...            66.0
...                   ...  ...             ...
2460              32056.0  ...            75.0
2461              32057.0  ...            73.0
2462              32058.0  ...           121.0
2463                  NaN  ...             NaN
2464                  NaN  ...             NaN

[2465 rows x 307 columns]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (92,93,131,132,301) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
Seq.head()

,Clave del Municipio,Nombre del Municipio,Entidad,2003-01-31,2003-02-28,2003-03-31,2003-04-30,2003-05-31,2003-06-30,2003-07-31,2003-09-30,2003-10-31,2003-11-30,2003-12-31,2004-01-31,2004-03-31,2004-04-30,2004-05-31,2004-06-30,2004-07-31,2004-08-31,2004-09-30,2004-10-31,2004-11-30,2004-12-31,2005-01-31,2005-02-28,2005-03-31,2005-04-30,2005-05-31,2005-06-30,2005-07-31,2005-08-31,2005-09-30,2005-10-31,2005-11-30,2005-12-31,2006-01-31,2006-02-28,2006-03-31,...,2019-08-31,2019-09-15,2019-09-30,2019-10-15,2019-10-31,2019-11-15,2019-11-30,2019-12-15,2019-12-31,2020-01-15,2020-01-31,2020-02-15,2020-02-29,2020-03-15,2020-03-31,2020-04-15,2020-04-30,2020-05-15,2020-05-31,2020-06-15,2020-06-30,2020-07-15,2020-07-31,2020-08-15,2020-08-31,2020-09-15,2020-09-30,2020-10-15,2020-10-31,2020-11-15,2020-11-30,2020-12-15,2020-12-31,Unnamed: 300,Anormalmente Seco (D0),Sequía Moderada (D1),Sequía Severa (D2),Sequía Extrema (D3),Sequía Excepcional (D4),Total (D0 a D4)
0,1001.0,Aguascalientes,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Severa,...,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Moderada,Sequía Moderada,NaN,33,22.0,4.0,5.0,1.0,65.0
1,1002.0,Asientos,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Moderada,...,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Moderada,Sequía Moderada,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Moderada,Sequía Moderada,NaN,33,19.0,8.0,4.0,0.0,64.0
2,1003.0,Calvillo,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Severa,...,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,NaN,NaN,Anormalmente Seco,NaN,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,NaN,38,20.0,7.0,4.0,2.0,71.0
3,1004.0,Cosío,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Moderada,...,Sequía Moderada,Sequía Moderada,Sequía Moderada,Sequía Moderada,Sequía Moderada,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,NaN,26,23.0,9.0,3.0,0.0,61.0
4,1005.0,Jesús María,Aguascalientes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Severa,...,Anormalmente Seco,Anormalmente Seco,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,NaN,NaN,NaN,NaN,NaN,NaN,Anormalmente Seco,Anormalmente Seco,Sequía Moderada,Sequía Moderada,Sequía Moderada,NaN,33,21.0,6.0,4.0,2.0,66.0


Se eliminan las columnas que no se utilizarán

In [4]:
df = Seq

df = df.drop(columns={"Clave del Municipio", "Nombre del Municipio", "Anormalmente Seco (D0)", "Sequía Moderada (D1)", "Sequía Severa (D2)", "Sequía Extrema (D3)", "Sequía Excepcional (D4)", "Total (D0 a D4)",  })

#Se elimina la columna vacía
df = df.drop(Seq.columns[299],axis=1)

Se renombran las clasificaciones de las sequías y se abrevian algunos estados 

In [5]:
#Se abrevian las clasificaciones de las sequías
df = df.replace(to_replace ="Anormalmente Seco",
                 value ="D0")
df = df.replace(to_replace ="Sequía Moderada",
                 value ="D1")
df = df.replace(to_replace ="Sequía Severa",
                 value ="D2")
df = df.replace(to_replace ="Sequía Extrema",
                 value ="D3")
df = df.replace(to_replace ="Sequía Excepcional",
                 value ="D4")

#Se abrevian los nombres de los estados
df = df.replace(to_replace = "Veracruz de Ignacio de la Llave",
                value = "Veracruz")
df = df.replace(to_replace = "Michoacán de Ocampo",
                value = "Michoacán")
df = df.replace(to_replace = "Coahuila de Zaragoza",
                value = "Coahuila")
df = df.replace(to_replace = "Querétaro de Arteaga",
                value = "Querétaro")


In [6]:
#Se reacomoda el dataframe, a modo de tener en una columna las fechas de las tormentas
df1 = (df.set_index(["Entidad"])
         .stack()
         .reset_index(name='Sequía')
         .rename(columns={'level_1':'Fecha'}))

#Se convierte a tipo fecha los valores en la columna de las fechas
df1['Fecha'] = pd.to_datetime(df1['Fecha'], errors='coerce')

#Se crea una nueva columna para tener por separado día, mes y año
df1['Día'] = df1['Fecha'].dt.day
df1['Mes'] = df1['Fecha'].dt.month
df1['Anio'] = df1['Fecha'].dt.year

#Se ordena el dataframe por fechas
df1 = df1.sort_values(by='Fecha')

df1 = df1.drop(columns={'Fecha', 'Día'})

print (df1.head(30))

                 Entidad Sequía  Mes  Anio
58487           Guerrero     D2    1  2003
149438            Oaxaca     D1    1  2003
149348            Oaxaca     D0    1  2003
38072   Ciudad de México     D0    1  2003
149228            Oaxaca     D0    1  2003
149095            Oaxaca     D1    1  2003
238738           Tabasco     D1    1  2003
149021            Oaxaca     D1    1  2003
38166            Durango     D0    1  2003
259088          Veracruz     D0    1  2003
148892            Oaxaca     D1    1  2003
238870        Tamaulipas     D0    1  2003
148570            Oaxaca     D1    1  2003
239007        Tamaulipas     D0    1  2003
148472            Oaxaca     D3    1  2003
38292            Durango     D1    1  2003
239181        Tamaulipas     D0    1  2003
148377            Oaxaca     D3    1  2003
240354        Tamaulipas     D1    1  2003
38572            Durango     D0    1  2003
147191            Oaxaca     D2    1  2003
240211        Tamaulipas     D0    1  2003
239968     

Se ordenan las filas de acuerdo a entidad, año, mes, y número de tormentas de acuerdo a su clasificación

In [8]:
anios = range(2003, 2021)
estados_comp = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de Mexico', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Estado de Mexico', 'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla', 'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas']

meses = list(range(1, 13))

df3 = pd.DataFrame(columns={"Entidad", "Anio", "Mes", "D0", "D1", "D2", "D3", "D4", "Total Sequías"})

l = 0
for i in anios:
  for j in meses:
    for k in estados_comp:
      df2 = df1[(df1['Entidad']==k) & (df1['Anio']==i) & (df1['Mes']==j)]

      df3.at[l, "Entidad"] = k
      df3.at[l, "Anio"] = i
      df3.at[l, "Mes"] = j

      #Se copia la cantidad de tormentas por clasificación que aparecen en la entidad, mes y año correspondientes
      df3.at[l, "D0"] = (df2.Sequía=='D0').sum().sum()
      df3.at[l, "D1"] = (df2.Sequía=='D1').sum().sum()
      df3.at[l, "D2"] = (df2.Sequía=='D2').sum().sum()
      df3.at[l, "D3"] = (df2.Sequía=='D3').sum().sum()
      df3.at[l, "D4"] = (df2.Sequía=='D4').sum().sum()

      #Se agrega un total de sequías, corespondiente a la suma de los cuatro tipos de sequías
      df3.at[l, "Total Sequías"] = df3.at[l, "D0"] + df3.at[l, "D1"] + df3.at[l, "D2"] + df3.at[l, "D3"] + df3.at[l, "D4"]

      l += 1

#Se reordenan las columnas del dataframe
df3 = df3[["Entidad", "Anio", "Mes", "D0", "D1", "D2", "D3", "D4", "Total Sequías"]]

print(df3)

                  Entidad  Anio Mes   D0  D1 D2 D3 D4 Total Sequías
0          Aguascalientes  2003   1    0   0  0  0  0             0
1         Baja California  2003   1    2   1  1  1  0             5
2     Baja California Sur  2003   1    1   0  1  0  0             2
3                Campeche  2003   1    6   0  0  0  0             6
4                Coahuila  2003   1    7   3  5  0  0            15
...                   ...   ...  ..  ...  .. .. .. ..           ...
6907           Tamaulipas  2020  12    9  27  3  3  0            42
6908             Tlaxcala  2020  12   28   0  0  0  0            28
6909             Veracruz  2020  12  130  24  0  0  0           154
6910              Yucatan  2020  12    0   0  0  0  0             0
6911            Zacatecas  2020  12   16  37  5  0  0            58

[6912 rows x 9 columns]


Se crea un nuevo dataframe que solo contiene los datos a nivel nacional y por año

In [24]:
anios = list(range(2003, 2021))

df4 = pd.DataFrame(columns={"Entidad", "Anio", "Mes", "D0", "D1", "D2", "D3", "D4", "Total_Sequías"})

j = 0
for i in anios:
  df = df3[df3["Anio"]==i]

  df4.at[j, "Entidad"] = "NAC"
  df4.at[j, "Anio"] = i
  df4.at[j, "Mes"] = "ANUAL"
  df4.at[j, "D0"] = (df.D0).sum().sum()
  df4.at[j, "D1"] = (df.D1).sum().sum()
  df4.at[j, "D2"] = (df.D2).sum().sum()
  df4.at[j, "D3"] = (df.D3).sum().sum()
  df4.at[j, "D4"] = (df.D4).sum().sum()
  df4.at[j, "Total_Sequías"] = (df["Total Sequías"]).sum().sum()

  j += 1

df4 = df4[["Entidad", "Anio", "Mes", "D0", "D1", "D2", "D3", "D4", "Total_Sequías"]]
print(df4)

   Entidad  Anio    Mes     D0    D1    D2    D3   D4 Total_Sequías
0      NAC  2003  ANUAL   6131  2625  1026   448   30         10260
1      NAC  2004  ANUAL   4358  1243    48    19    5          5673
2      NAC  2005  ANUAL   6096  1954    89     1    0          8140
3      NAC  2006  ANUAL   6218  2941  1401   486  214         11260
4      NAC  2007  ANUAL   4427  2725   910   201    4          8267
5      NAC  2008  ANUAL   6217  3209   971   199    9         10605
6      NAC  2009  ANUAL   6296  4697   953   132    8         12086
7      NAC  2010  ANUAL   2365   888   864   312    0          4429
8      NAC  2011  ANUAL   4195  3376  2805  1987  500         12863
9      NAC  2012  ANUAL   2032  1434   760   462   82          4770
10     NAC  2013  ANUAL   4150   765   157    61   19          5152
11     NAC  2014  ANUAL   7809  1297   275    54    0          9435
12     NAC  2015  ANUAL   9266  2646   392    64    0         12368
13     NAC  2016  ANUAL  12287  6239  1556   304

Se lee el archivo que contiene los datos de las temperaturas y nivel de precipitación

In [10]:
url = "https://github.com/AdrianDiazGaxiola/Proyecto-Final/blob/main/Data/DataFrames%20Combinados/Temperaturas+Precipitaci%C3%B3n.csv?raw=true"
df_completo = pd.read_csv(url)

Se combina el dataframe que incluye las temperaturas y nivel de precipitación, tomando solamente los datos por año y a nivel nacional 

In [33]:
df = df_completo[(df_completo["ANIO"]>=2003) & (df_completo["ENTIDAD"]=="NAC") & (df_completo["MES"]=="ANUAL")]

d0 = df4["D0"].tolist()
d1 = df4["D1"].tolist()
d2 = df4["D2"].tolist()
d3 = df4["D3"].tolist()
d4 = df4["D4"].tolist()
total = df4["Total_Sequías"].tolist()

df["D0"] = d0
df["D1"] = d1
df["D2"] = d2
df["D3"] = d3
df["D4"] = d4
df["Total_Sequías"] = total

df = df.reset_index(drop=True)
print(df.head(5))


   ANIO    MES ENTIDAD      T_Max  ...    D2   D3   D4  Total_Sequías
0  2003  ANUAL     NAC  28.643543  ...  1026  448   30          10260
1  2004  ANUAL     NAC  28.345458  ...    48   19    5           5673
2  2005  ANUAL     NAC  28.967507  ...    89    1    0           8140
3  2006  ANUAL     NAC  29.215505  ...  1401  486  214          11260
4  2007  ANUAL     NAC  29.348220  ...   910  201    4           8267

[5 rows x 13 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

Se descarga el nuevo dataframe con los datos adjuntos

In [34]:
df.to_csv("Temp+Prec+Seq.csv", encoding='utf-8', index=False)
from google.colab import files
files.download("Temp+Prec+Seq.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
seq_por_entidad = {}

anios = range(2003, 2021)
estados_comp = ['Aguascalientes', 'Baja California', 'Baja California Sur', 'Campeche', 'Coahuila', 'Colima', 'Chiapas', 'Chihuahua', 'Ciudad de Mexico', 'Durango', 'Guanajuato', 'Guerrero', 'Hidalgo', 'Jalisco', 'Estado de Mexico', 'Michoacan', 'Morelos', 'Nayarit', 'Nuevo Leon', 'Oaxaca', 'Puebla', 'Queretaro', 'Quintana Roo', 'San Luis Potosi', 'Sinaloa', 'Sonora', 'Tabasco', 'Tamaulipas', 'Tlaxcala', 'Veracruz', 'Yucatan', 'Zacatecas']

#Ciclo con una iteración por estado de la república
for i in range (32):
  #Se guarda en un dataframe aparte los datos del estado en el que nos encontramos
  df1 = Seq[Seq['Entidad']==estados_comp[i]]

  #Se establecen las columnas del nuevo dataframe que contendrá los datos del estado en el que nos ubicamos, incluyendo año y la cantidad de sequías por clasificación de cada año.
  df2 = pd.DataFrame(columns=['ANIO', 'Anormalmente Seco', 'Sequia Moderada', 'Sequia Severa', 'Sequia Extrema', 'Sequia Excepcional', 'Total'])

  for j in range (2003,2021):
    l = j - 2002

    #Se guarda el número del año en la fila en que nos encontramos, debajo de la columna 'ANIO'
    df2.at[l, 'ANIO'] = anios[l-1]

    #Se suma independientemente la cantidad de sequías de acuerdo a su clasificación en el año en que nos encontramos, utilizando la función df.filter ya que tenemos varias columnas cuyos nombres (los años) se repiten.
    df2.at[l, 'Anormalmente Seco'] = (df1.filter(like = str(j))=='D0').sum().sum()
    df2.at[l, 'Sequia Moderada'] = (df1.filter(like = str(j))=='D1').sum().sum()
    df2.at[l, 'Sequia Severa'] = (df1.filter(like = str(j))=='D2').sum().sum()
    df2.at[l, 'Sequia Extrema'] = (df1.filter(like = str(j))=='D3').sum().sum()
    df2.at[l, 'Sequia Excepcional'] = (df1.filter(like = str(j))=='D4').sum().sum()

    #Se obtiene el valor de la columna 'Total' sumando los valores obtenidos en el paso anterior
    df2.at[l, 'Total'] = df2.at[l, 'Anormalmente Seco'] + df2.at[l, 'Sequia Moderada'] + df2.at[l, 'Sequia Severa'] + df2.at[l, 'Sequia Extrema'] + df2.at[l, 'Sequia Excepcional'] 

  print(estados_comp[i])
  print(df2)        

#Se crea un dataframe con los datos de las sequías por año a nivel NACIONAL
df3 = pd.DataFrame(columns=['ANIO', 'Anormalmente Seco', 'Sequia Moderada', 'Sequia Severa', 'Sequia Extrema', 'Sequia Excepcional', 'Total'])
for i in range (2003, 2021):
  j = i - 2002

  df3.at[j, 'ANIO'] = anios[j-1]

  df3.at[j, 'Anormalmente Seco'] = (Seq.filter(like = str(i))=='D0').sum().sum()
  df3.at[j, 'Sequia Moderada'] = (Seq.filter(like = str(i))=='D1').sum().sum()
  df3.at[j, 'Sequia Severa'] = (Seq.filter(like = str(i))=='D2').sum().sum()
  df3.at[j, 'Sequia Extrema'] = (Seq.filter(like = str(i))=='D3').sum().sum()
  df3.at[j, 'Sequia Excepcional'] = (Seq.filter(like = str(i))=='D4').sum().sum()
  df3.at[j, 'Total'] = df3.at[j, 'Anormalmente Seco'] + df3.at[j, 'Sequia Moderada'] + df3.at[j, 'Sequia Severa'] + df3.at[j, 'Sequia Extrema'] + df3.at[j, 'Sequia Excepcional']

print ("NACIONAL")
print (df3)


Aguascalientes
    ANIO Anormalmente Seco  ... Sequia Excepcional Total
1   2003                 0  ...                  0     0
2   2004                 0  ...                  0     0
3   2005                22  ...                  0    22
4   2006                27  ...                  0    77
5   2007                29  ...                  0    33
6   2008                12  ...                  0    53
7   2009                31  ...                  0    44
8   2010                 0  ...                  0     0
9   2011                 5  ...                  5   110
10  2012                30  ...                  0    73
11  2013                17  ...                  0    17
12  2014                30  ...                  0    30
13  2015                 0  ...                  0     0
14  2016                38  ...                  0    38
15  2017                77  ...                  0   132
16  2018                 0  ...                  0     0
17  2019        